In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First, prep the candidates:

In [2]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/hardware_docs/')

In [3]:
from snorkel.parser import TableParser
table_parser = TableParser()

In [4]:
from snorkel.parser import Corpus
%time corpus = Corpus(html_parser, table_parser, max_docs=1000)

Parsing documents...
Parsing contexts...
Parsed 98 documents and 630 contexts
CPU times: user 1min 18s, sys: 5.13 s, total: 1min 23s
Wall time: 2min 6s


In [20]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import NumberMatcher, RangeMatcher

table_ngrams = TableNgrams(n_max=3)
number_matcher = RangeMatcher(low=-80,high=-40)

In [21]:
from snorkel.candidates import Candidates
%time candidates = Candidates(table_ngrams, number_matcher, corpus.get_contexts())

Extracting candidates...
CPU times: user 5.67 s, sys: 218 ms, total: 5.89 s
Wall time: 5.81 s


Next, prep the gold data:

In [22]:
import csv
with open('data/gold_stg_temp_min.csv', 'rb') as csvfile:
    gold_reader = csv.reader(csvfile)
    gold = []
    for row in gold_reader:
        (doc, part, temp, label) = row
        if label=='stg_temp_min':
            gold.append((doc,temp))
validation = 0
train = 1 - validation
gold = list(set(gold))
gold_validation = gold[int(len(gold)*train):]
gold_train = gold[:int(len(gold)*(train))]
print "gold_train: %s annotations" % len(gold_train)
print "gold_validation: %s annotations" % len(gold_validation)
print gold_train[0]

gold_train: 98 annotations
gold_validation: 0 annotations
('PHGLS20125-1', '-65')


In the end, we want a dictionary with {candidate UID : boolean}

In [23]:
gt_dict = {}
for c in candidates.get_candidates():
    filename = (corpus.get_doc(c.doc_id).file).split('.')[0]
    temp = c.get_attrib_span('words')
    label = 1 if (filename, temp) in gold_train else -1
    gt_dict[c.uid] = label
print gt_dict.items()[0]
print "%s out of %s candidates are True" % (gt_dict.values().count(1), len(gt_dict))

('71-0-10-0:0-2', -1)
63 out of 163 candidates are True


In [24]:
print len(gt_dict)

163


Save the results as a pickle

In [19]:
import cPickle
pkl_f = 'data/hardware_gold_train.pkl'
with open(pkl_f, 'w+') as f:
    cPickle.dump(gt_dict, f)